In [ ]:
!pip3 install --upgrade pip
!pip3 install --upgrade transformers
!pip3 install --upgrade accelerate
!pip3 install sentencepiece
!pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade
!pip install ipywidgets
!pip install peft
!pip install bitsandbytes
!pip install evaluate

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia

In [ ]:
from datasets import load_dataset
import json
from random import randrange
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, prepare_model_for_kbit_training, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import BitsAndBytesConfig

## Inference

In [ ]:
dataset = load_dataset('wepolyu/100qQALD')

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
file_path = '/content/prepared_data_erik.json'

dataset = load_dataset('json', data_files=file_path)

# Anzeigen des Datensets
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'triples_number', 'contexts', 'answer'],
        num_rows: 77
    })
})


In [ ]:
prompt_template="""[INST]
Utilize the provided context information to generate answers to the following question:
- Generate the answer to the question based on the provided context.
- Avoid including any explanations or apologies in your responses.
- Only respond to questions that ask for the answer to the target question without any additional information.
- Strictly provide the answer without any additional text.
Question: {question}
Context: {context}
[/INST] Answer:"""

In [ ]:
# prompt: import dataset rausch/prepared_data_file_zero_shot_prompting_100Q_4_experiments.json from huggle face
import json
import configparser
import os
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

max_length_input=6000
max_output_length = 50
chars_per_token = 4
max_length_chars = max_length_input * chars_per_token

def load_model(model_id):
    """
    Load the model and tokenizer for the specified model ID.

    Args:
        model_id (str): The ID of the model to load.

    Returns:
        model (transformers.PreTrainedModel): The loaded model.
        tokenizer (transformers.PreTrainedTokenizer): The loaded tokenizer.
    """
    # If a GPU is available, adjust the max memory settings
    if torch.cuda.is_available():
        # Get the current CUDA device id
        device_id = torch.cuda.current_device()
        # Get the total memory of the current CUDA device
        total_memory = torch.cuda.get_device_properties(device_id).total_memory
        # Convert to GB and subtract a small buffer to avoid out-of-memory issues
        max_memory = f'{int(total_memory/1024**3)-2}GB'
    else:
        # Set a default max memory to use in case there is no GPU
        max_memory = '300GB'

    # Prepare device map and max memory settings for each device
    device_map = 'auto'  # Let the library manage device placement automatically
    maxmem = {device_id: max_memory}

    # Load the model with the specified settings
    model = AutoModelForCausalLM.from_pretrained(model_id,
                                                 load_in_8bit=False,
                                                 torch_dtype=torch.float16,
                                                 device_map=device_map,
                                                 max_memory=maxmem)
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    # Set the padding token to the end-of-sequence token
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'right'

    return model, tokenizer

def zero_shot_prompting_local(model, tokenizer, example, context_type, prompt_template):
    """
    Generate zero-shot predictions using a local model.

    Args:
        model (transformers.PreTrainedModel): The loaded model.
        tokenizer (transformers.PreTrainedTokenizer): The loaded tokenizer.
        example (dict): The example containing the question and context.
        context_type (str): The type of context to use from the example.
        prompt_template (str): The prompt template to format the question and context.

    Returns:
        dict: A dictionary containing the role and content of the response.
    """
    context = clean_context(example["contexts"][context_type])
    truncated_context = truncate_context_to_max_chars(context, example['question'], prompt_template)
    formatted_prompt = prompt_template.format(question=example['question'], context=truncated_context)

    inputs = tokenizer(formatted_prompt, add_special_tokens=True, max_length=max_length_input, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(inputs, max_new_tokens=max_output_length)
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return response_text

def zero_shot_prompting(model, tokenizer, example, context_type, prompt_template):
    """
    Generate zero-shot predictions using a local model.

    Args:
        model (transformers.PreTrainedModel): The loaded model.
        tokenizer (transformers.PreTrainedTokenizer): The loaded tokenizer.
        example (dict): The example containing the question and context.
        context_type (str): The type of context to use from the example.
        prompt_template (str): The prompt template to format the question and context.

    Returns:
        dict: A dictionary containing the role and content of the response.
    """
    context = clean_context(example["contexts"])
    truncated_context = truncate_context_to_max_chars(context, example['question'], prompt_template)
    formatted_prompt = prompt_template.format(question=example['question'], context=truncated_context)

    inputs = tokenizer(formatted_prompt, add_special_tokens=True, max_length=max_length_input, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(inputs, max_new_tokens=max_output_length)
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return response_text

def truncate_context_to_max_chars(context, question, prompt_template):
    """
    Truncate the context to ensure it fits within the maximum character length.

    Args:
        context (str): The context to truncate.
        question (str): The question for the prompt.
        prompt_template (str): The prompt template.

    Returns:
        str: The truncated context.
    """
    formatted_prompt = prompt_template.format(question=question, context='')
    prompt_length = len(formatted_prompt)
    max_context_length = max_length_chars - prompt_length

    if len(context) > max_context_length:
        print(f"Original context is too long and will be truncated. Original length: {len(context)}, truncated length: {max_context_length}")
        context = context[:max_context_length]

    return context

def clean_context(context):
    """
    Clean the context by removing special characters.

    Args:
        context (str): The context to be cleaned.

    Returns:
        str: The cleaned context.
    """
    return context.replace('<unk>', ' ')


In [ ]:
from huggingface_hub import HfApi

api = HfApi()

def run_model(model_id, model_name, templates,prepared_data):
    """
    Run the model to generate zero-shot predictions for the given templates.

    Args:
        model_id (str): The ID of the model to load.
        model_name (str): The name of the model.
        template_files (list): A list of template file paths.
        api (bool, optional): Whether to use an API. Defaults to False.
        api_type (str, optional): The type of API to use ('llama' or 'groq'). Defaults to None.
    """
    if(model_name=='T5'):
      tokenizer = AutoTokenizer.from_pretrained(model_id)
      model=AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
    else:
      tokenizer = AutoTokenizer.from_pretrained(model_id)
      model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
    results = []
    for example in tqdm(prepared_data, desc=f"Generating Zero-Shot Predictions for {model_name}"):
        print(example,"hello")

        result = {
            "id": example["id"],
            "question": example["question"],
            # "true_answer": example["answer"],
            "model": model_name,
            "technique": "zero_shot",
            "results": []
        }

        for prompt_template in templates:
            # Remove comments and whitespace from the file list
            response = zero_shot_prompting(model, tokenizer, example, " ", prompt_template)
            result['answer']=response

        results.append(result)
    with open("result.json", "w") as json_file:
      temp=json.dump(results, json_file, indent=4, ensure_ascii=False)

    api.upload_file(
    path_or_fileobj="result.json",
    path_in_repo="results_1.json",
    repo_id="wepolyu/KGQA",
    repo_type="dataset",
)


    # Clean up the model and tokenizer to free up memory
    if model:
        del model
    if tokenizer:
        del tokenizer
    torch.cuda.empty_cache()

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

def run_model4(model_id, model_name, templates,prepared_data):
    """
    Run the model to generate zero-shot predictions for the given templates.

    Args:
        model_id (str): The ID of the model to load.
        model_name (str): The name of the model.
        template_files (list): A list of template file paths.
        api (bool, optional): Whether to use an API. Defaults to False.
        api_type (str, optional): The type of API to use ('llama' or 'groq'). Defaults to None.
    """
    if(model_name=='T5'):
      tokenizer = AutoTokenizer.from_pretrained(model_id)
      model=AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")
    else:
      tokenizer = AutoTokenizer.from_pretrained(model_id)
      model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
    results = []
    for example in tqdm(prepared_data, desc=f"Generating Zero-Shot Predictions for {model_name}"):
        result = {
            "id": example["id"],
            "question": example["question"],
            # "true_answer": example["answer"],
            "model": model_name,
            "technique": "zero_shot",
            "results": []
        }

        for prompt_template in templates:
            # Remove comments and whitespace from the file list
            template_result = {
                "prompt_template": prompt_template,
                "all_triples_results": {
                    "response": {},
                    "metrics": {
                        "exact_score": 0,
                        "meteor": 0
                    }
                },
                "verbalizer_results": {
                    "response": {},
                    "metrics": {
                        "exact_score": 0,
                        "meteor": 0
                    }
                },
                "evidence_matching": {
                    "response": {},
                    "metrics": {
                        "exact_score": 0,
                        "meteor": 0
                    }
                },
                "verbalizer_plus_evidence_matching": {
                    "response": {},
                    "metrics": {
                        "exact_score": 0,
                        "meteor": 0
                    }
                }
            }

            for context_type, context_key in zip(
                ["plain", "verbalizer_on_all_triples", "evidence_matching", "verbalizer_plus_evidence_matching"],
                ["all_triples_results", "verbalizer_results", "evidence_matching", "verbalizer_plus_evidence_matching"]
            ):

                response = zero_shot_prompting_local(model, tokenizer, example, context_type, prompt_template)

                template_result[context_key]["response"] = response
            # response = zero_shot_prompting_local(model, tokenizer, example, "", prompt_template)
            # template_result["response"] = response
            result["results"].append(template_result)

        results.append(result)
    with open("result.json", "w") as json_file:
      temp=json.dump(results, json_file, indent=4, ensure_ascii=False)

    api.upload_file(
    path_or_fileobj="result.json",
    path_in_repo="results_1.json",
    repo_id="wepolyu/KGQA",
    repo_type="dataset",
)


    # Clean up the model and tokenizer to free up memory
    if model:
        del model
    if tokenizer:
        del tokenizer
    torch.cuda.empty_cache()

In [ ]:
# run_model("meta-llama/Meta-Llama-3-8B", "Llama-3-8B", [prompt_template],[dataset["train"][0]])
run_model4("google/flan-t5-large", "T5", [prompt_template],dataset['train'])

Generating Zero-Shot Predictions for T5:   0%|          | 0/77 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original context is too long and will be truncated. Original length: 137163, truncated length: 23415


Generating Zero-Shot Predictions for T5:   1%|▏         | 1/77 [00:05<07:04,  5.59s/it]

Original context is too long and will be truncated. Original length: 90454, truncated length: 23429


Generating Zero-Shot Predictions for T5:   3%|▎         | 2/77 [00:07<04:19,  3.46s/it]

...

Original context is too long and will be truncated. Original length: 864126, truncated length: 23414


Generating Zero-Shot Predictions for T5:  99%|█████████▊| 76/77 [04:12<00:03,  3.58s/it]

Original context is too long and will be truncated. Original length: 93060, truncated length: 23373


Generating Zero-Shot Predictions for T5: 100%|██████████| 77/77 [04:16<00:00,  3.33s/it]


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-66aed5cf-47a3f6e21af3fd1510afebaf;c8bdf8cb-5adb-44af-9431-53279a6d812e)

Repository Not Found for url: https://huggingface.co/api/datasets/wepolyu/KGQA/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.

In [ ]:
dataset["train"][20]

In [ ]:
# with open("result.json", "r") as json_file:
#     data = json.load(json_file)

In [ ]:
data[0]

In [ ]:
# results=[]
# for item in data:
#   results.append({"answer":item["results"][0]["response"],"id":item["id"]})

In [ ]:
# with open("result_3.json", "w") as json_file:
#     json.dump(results,json_file,indent=2)

In [ ]:
import torch
torch.cuda.empty_cache()